# Automatic differentiation with ``autograd`` 


In machine learning, we want models to get better and better as a function of experience. Usually *getting better*, means minimizing a *loss function*, a measure of how *bad* our model is at any time. With neural networks, that loss is usually differentiable, i.e. for each of the model's parameters, we can always determine how much increasing or decreasing it might affect the loss. For complex models, working out these derivatives from scratch can be a pain.

_MXNet_'s autograd package eliminates this tedious work by automatically calculating derivatives for you. Other libraries require you to pre-define and compile symbolic graphs in order to access automatic derivatives. However, ``autograd``, much like the similar package in PyTorch, allows you to take derivatives even when running fully imperative code.

Essentially, every time you make pass through your model, autograd builds a graph on the fly, through which it can immediately backpropagate gradients.

Let's go through it step by step. For this tutorial, we'll only need to import ``mxnet.ndarray``, and ``mxnet.autograd``.

In [3]:
import mxnet as mx
import mxnet.ndarray as nd
from mxnet import autograd

## Attaching Gradients

As a toy example, Let's say that we are interested in differentiating a function ``f = 2 * (x ** 2)`` with respect to parameter x. We can start by assining an initial value of ``x``.

In [14]:
x = mx.nd.array([[1, 2], [3, 4]])

Once we compute the gradient of ``f`` with respect to ``x``, we'll need a place to store it. In _MXNet_, we can tell an NDArray that we plan to store a gradient by invoking its ``atach_grad()`` method.

In [15]:
x.attach_grad()

Now we're going to define out function ``f`` and *MXNet* will generate a computation graph on the fly. It's as if *MXNet* turned on a recording device and captured the exact path by which each variabel was generated. 

Note that building the computation graph requires a nontrivial amount of computation. So we only *MXNet* to build the graph when explicitlt told to do so. We can instruct *MXNet* to start recording by placing code inside a ``with autograd.record():`` block.

In [16]:
with autograd.record():
  y = x * 2
  f = y * x

Let's backprop with f.backward(). When f has more than one entry, f.backward() is equivalent to mx.nd.sum(f).backward().



In [17]:
f.backward()

Now, let's see if this is the expected output. Remember that ``y = x * 2``, and ``f = x * y``, so ``f`` should be equal to  ``2 * x * x``. After, doing backprop with ``f.backward()``, we expect to get back gradient df/dx as follows: dy/dx = ``2``, df/dx = ``4 * x``. So, if everything went according to plan, ``x.grad`` should consist of an NDArray with the values ``[[4, 8],[12, 16]]``.

In [8]:
print(x.grad)

[[ 4.          8.        ]
 [ 1.20000005  1.60000002]]
<NDArray 2x2 @cpu(0)>


## Head Gradients and the Chain Rule

*Warning: This part is tricky, but not necessary to understanding subsequent sections.*

Sometimes when we call the backward method on an NDArray, e.g. ``y.backward()``, where ``y`` is a function of ``x`` we are just interested in the derivative of ``y`` with respect to ``x``. At other times, we may be interested in the gradient of ``z`` with respect to ``x``, where ``z`` is a function of ``y``. Recall that by the train rule dz/dx can be expressed in terms of dz/dy and dy/dx. So, when ``y`` is part of a larger function ``z``, and we want ``x.grad`` to store dz/dx, we can pass in the *head gradient* dz/dy as an input to ``backward()``. 

In [21]:
with autograd.record():
  y = x * 2
  f = y * x
    
gradients = nd.array([[10,1.],[.1,.01]])
f.backward(gradients)
print(x.grad)

[[ 40.           8.        ]
 [  1.20000005   0.16      ]]
<NDArray 2x2 @cpu(0)>


Now that we know the basics, we can do some wild things with autograd, including building diferentiable functions using Pythonic control flow.

In [9]:
a = nd.random_normal(shape=3)
a.attach_grad()

with autograd.record():
    b = a * 2
    while (nd.norm(b) < 1000).asscalar():
        b = b * 2

    if (mx.nd.sum(b) > 0).asscalar():
        c = b
    else :
        c = 100 * b

In [12]:
gradients = nd.array([0.01,1.0,.1])
c.backward(gradients)

MXNetError: [20:02:50] src/ndarray/autograd.cc:157: Check failed: !i.entry_.is_none() Cannot differentiate node because it is not in a computational graph. You need to set is_training to true or use autograd.record() to save computational graphs for backward. If you want to differentiate the same graph twice, you need to pass retain_graph=True to backward.

Stack trace returned 5 entries:
[bt] (0) 0   libmxnet.so                         0x0000000108e458a5 _ZN4dmlc15LogMessageFatalD2Ev + 37
[bt] (1) 1   libmxnet.so                         0x000000010967f59b _ZN5mxnet8autograd15AutogradRuntime15ComputeGradientERKNSt3__16vectorINS_7NDArrayENS2_9allocatorIS4_EEEES9_b + 11131
[bt] (2) 2   libmxnet.so                         0x00000001095b873e MXAutogradBackward + 494
[bt] (3) 3   _ctypes.cpython-36m-darwin.so       0x00000001032412c7 ffi_call_unix64 + 79
[bt] (4) 4   ???                                 0x00007fff5e460560 0x0 + 140734775035232


In [11]:
print(x.grad)

[[ 4.          8.        ]
 [ 1.20000005  1.60000002]]
<NDArray 2x2 @cpu(0)>
